# Collecting Player Dataset
In this notebook, we will
1. web scrape player data from fbref.com
2. Map Euro Fantasy Player list to FBRef player list

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
from pathlib import Path
from unidecode import unidecode

# disable pandas warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Scraping player table from FBRef using BeautifulSoup

### 1.1 Get player by "part of name"
We can search for players by the first 2 letters of their last names

In [2]:
def extract_player_data_from_p(player_tag):
    """
    Extract player data from a p tag from fbref player page
    """
    # get player name which is the text in a element
    a_elem = player_tag.find('a')
    player_name = a_elem.text
    player_url = a_elem['href']

    player_active = bool(a_elem.find_all('strong')) # player is active if name is bold

    span_elem = player_tag.find('span') # try to get country in span element
    if span_elem:
        player_country = span_elem.text.upper()
        
        # get all text in the after the span element
        text_after_span = player_tag.find_all("span")[-1].next_sibling
        player_info = text_after_span.get_text().lstrip(' \xa0·').split('\xa0· ')
    else:
        # no span element found, which means no country code
        player_country = None
        # just get all text in the p element after the player name
        other_info = a_elem.next_sibling
        player_info = other_info.get_text().strip().split('\xa0· ')
    player_info.extend([None] * (3 - len(player_info))) # ensure player_info is of length 3
    
    return player_name, player_country, player_url, player_active, *player_info

In [3]:
def url_builder_player_name(keyword):
    """ Return FBRef URL based on competition name
    """
    if len(keyword) != 2:
        raise ValueError("Keyword must be 2 characters long")
    
    scheme_domain_language_str = "https://fbref.com/en"
    url = f"{scheme_domain_language_str}/players/{keyword}/"
    return url

def get_player_by_part_of_name(keyword: str):
    """
    Get player data by first two letters of their name (usually last name)
    """    
    url = url_builder_player_name(keyword)
    raw_html = requests.get(url).text
    soup = BeautifulSoup(raw_html, "html.parser") # parse the html using BeautifulSoup
    content = soup.find("div", class_="section_content", id=re.compile("div_*")) # get the main content div
    info_lst = [extract_player_data_from_p(row) for row in content.find_all("p")] # extract player data from p tags

    COLNAMES = ['Name', 'country_code_alpha2', 'url', 'Active', 'Years Played', 'Position', 'Clubs Played']
    df = pd.DataFrame(info_lst, columns=COLNAMES)
    return df

Example usage

In [4]:
df = get_player_by_part_of_name("mb")
df.head()

,Name,country_code_alpha2,url,Active,Years Played,Position,Clubs Played
0,Joshua M'Bahia,CI,/en/players/f02067df/Joshua-MBahia,True,2023-2024,DF,Clermont Foot
1,Salem M'bakata,CD,/en/players/0c75df2a/Salem-Mbakata,True,2018-2024,DF,"Gaziantep FK, Sochaux, Aris"
2,Tatenda M'balaka,MW,/en/players/f64c1f4b/Tatenda-Mbalaka,True,2023-2026,DF,Malawi
3,Modeste M'bami,CM,/en/players/82ec2963/Modeste-Mbami,False,2000-2016,MF,"Marseille, Paris Saint-Germain, Sedan, Almería..."
4,Bryan M'Bango,None,/en/players/db91d5d7/Bryan-MBango,False,2009-2010,None,None


Diacritics/accents in some player names make it difficult to match by default. For example, Kylian Mbappe

In [5]:
# get mbappe's data based on regex
df[df['Name'].str.contains('K.+ Mbapp.', case=False, regex=True)]

,Name,country_code_alpha2,url,Active,Years Played,Position,Clubs Played
88,Kylian Mbappé,FR,/en/players/42fd9c7f/Kylian-Mbappe,True,2015-2024,"FW,MF","Paris Saint-Germain, France, Monaco"


### 1.2 Get all players from a country

In [102]:
def url_builder_country_players(keyword):
    """ Return FBRef URL based on competition name
    """
    if len(keyword) != 3:
        raise ValueError("Country Keyword must be 3 characters long")
    
    scheme_domain_language_str = "https://fbref.com/en"
    url = f"{scheme_domain_language_str}/country/players/{keyword}/Football-Players" # fbref redirects it correctly
    return url

def get_players_from_country(country_ioc_code: str):
    """
    Get all player data from a country
    """    
    url = url_builder_country_players(country_ioc_code)
    raw_html = requests.get(url).text

    soup = BeautifulSoup(raw_html, "html.parser") # parse the html using BeautifulSoup
    content = soup.find("div", class_="section_content", id=re.compile("div_*")) # get the main content div
    info_lst = [extract_player_data_from_p(row) for row in content.find_all("p")] # extract player data from p tags

    COLNAMES = ['Name', 'country_code_alpha2', 'url', 'Active', 'Years Played', 'Position', 'Clubs Played']
    df = pd.DataFrame(info_lst, columns=COLNAMES)
    
    df.drop(columns=['country_code_alpha2'], inplace=True) #TODO: country_code_alpha2 column can be removed
    df['IOC'] = country_ioc_code.upper()

    return df

Example usage

In [106]:
fra_df = get_players_from_country("FRA")
fra_active_df = fra_df[fra_df['Active']] # get active players only

In [104]:
fra_active_df.head()

,Name,url,Active,Years Played,Position,Clubs Played,IOC
1,Rayane Aabid,/en/players/13792bac/Rayane-Aabid,True,2017-2024,MF,"Hatayspor, Béziers, Yeni Malatyaspor, Kasımpaş...",FRA
6,Ismael Aaneba,/en/players/a7e4760c/Ismael-Aaneba,True,2017-2024,DF,"Ferencváros, Sochaux, Strasbourg",FRA
7,Othman Aankour,/en/players/af2fb5a0/Othman-Aankour,True,2020-2024,DF,AS Furiani-Agliani,FRA
10,Fares Aatout,/en/players/ad48bc2a/Fares-Aatout,True,2021-2024,MF,FC Bourgoin-Jallieu,FRA
14,Aymen Abahousse,/en/players/5377a3c2/Aymen-Abahousse,True,2023-2024,DF,Jarville JF,FRA


In [108]:
print(fra_df.shape)
print(fra_active_df.shape)

(15024, 7)
(5826, 7)


## 2. Map Euro Fantasy Player data to FBRef Player data

### 2.1 Map on player name (part of name)
read Euro Fantasy player data and add regex for each player name

In [2]:
fe_df = pd.read_csv("../data/clean/euro_fantasy_players.csv")
fe_df.reset_index(names=['player_id'], inplace=True)
fe_df[['fname_init', 'lname']] = fe_df['Players'].str.split(' ', n=1, expand=True) # split Player column by first space
fe_df['lname'] = fe_df['lname'].fillna('') # replace lname column which is na with empty string

# create regex pattern based on first name and last name if last name is not empty
lname_empty_filt = fe_df['lname'] == ''
fe_df.loc[lname_empty_filt, 'regex_pattern'] = fe_df['fname_init'] # if last name is empty, use only first name
fe_df.loc[~lname_empty_filt, 'regex_pattern'] = fe_df['fname_init'] + "+ " + fe_df['lname']

fe_df['regex_pattern'] = fe_df['regex_pattern'].apply(lambda x: unidecode(x)) # remove accents
fe_df['keyword'] = fe_df['regex_pattern'].apply(lambda x: x.lower().split()[-1][:2]) # get first two letters of last word in regex pattern
fe_df.head()

,player_id,Players,Price,Total pts,Selected,MD pts,Pts per €,Pts per MD,PotM pts,Goals,...,Mins played,Trans in,Trans out,img_url,Next Match,Position,fname_init,lname,regex_pattern,keyword
0,0,K. Mbappé,11.0,0.0,76.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9715.0,2632.0,https://img.uefa.com/imgml/TP/players/3/2024/3...,AUT v FRA,FWD,K.,Mbappé,K.+ Mbappe,mb
1,1,H. Kane,11.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10142.0,4501.0,https://img.uefa.com/imgml/TP/players/3/2024/3...,SRB v ENG,FWD,H.,Kane,H.+ Kane,ka
2,2,C. Ronaldo,10.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3962.0,5013.0,https://img.uefa.com/imgml/TP/players/3/2024/3...,POR v CZE,FWD,C.,Ronaldo,C.+ Ronaldo,ro
3,3,J. Bellingham,9.5,0.0,50.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7235.0,6766.0,https://gaming.uefa.com/en/eurofantasy/static-...,SRB v ENG,MID,J.,Bellingham,J.+ Bellingham,be
4,4,K. De Bruyne,9.5,0.0,21.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5474.0,6343.0,https://gaming.uefa.com/en/eurofantasy/static-...,BEL v SVK,MID,K.,De Bruyne,K.+ De Bruyne,br


In [ ]:
# players with one name only
# fe_df.iloc[[57, 83, 110, 148, 153, 230, 351, 424, 443, 454, 586, 589, 626, 928]]

It is difficult to map using player name only since data from euro fantasy data, `fe_df` does not have country data.
- one option is to collect list of players from each Euro 2024 country, then for each row, see if the player is found in that country.

### 2.2 Players from Countries in Euro 2024
We get the list of players who are from countries that are participating in Euro 2024.

In [5]:
euro24_country_lst = ['Albania', 'Austria', 'Belgium', 'Croatia', 'Czechia', 'Denmark', 'England', 'France', 'Georgia', 'Germany', 'Hungary', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Scotland', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Switzerland', 'Turkey', 'Ukraine']

country_iso_df = pd.read_html("https://www.worlddata.info/countrycodes.php", flavor='bs4')[0]
country_iso_df = country_iso_df[['Country',	'ISO 3166-1 alpha2', 'ISO 3166-1 alpha3', 'IOC']].rename(columns={'ISO 3166-1 alpha2': 'alpha2', 'ISO 3166-1 alpha3': 'alpha3'})
euro24_country_df = country_iso_df[country_iso_df['Country'].isin(euro24_country_lst)]

# add rows for england and scotland
temp_df = pd.DataFrame([
    {'Country': 'England', 'alpha2': 'ENG', 'alpha3': 'ENG', 'IOC': 'ENG'},
    {'Country': 'Scotland', 'alpha2': 'SCT', 'alpha3': 'SCO', 'IOC': 'SCO'}
])

euro24_country_df.loc[euro24_country_df['Country'] == 'Slovenia', 'IOC'] = 'SVN' # replace IOC value for Slovenia

euro24_country_df = pd.concat([euro24_country_df, temp_df], ignore_index=True).sort_values('Country').reset_index(drop=True)

In [156]:
euro24_country_df

,Country,alpha2,alpha3,IOC
0,Albania,AL,ALB,ALB
1,Austria,AT,AUT,AUT
2,Belgium,BE,BEL,BEL
3,Croatia,HR,HRV,CRO
4,Czechia,CZ,CZE,CZE
5,Denmark,DK,DNK,DEN
6,England,ENG,ENG,ENG
7,France,FR,FRA,FRA
8,Georgia,GE,GEO,GEO
9,Germany,DE,DEU,GER


In [157]:
def collect_active_players_in_countries(country_lst):
    main_dir = Path.cwd().resolve().parents[0]

    for cnt_code in country_lst:
        print(f"Collecting player data for country {cnt_code}")
        try:
            curr_df = get_players_from_country(cnt_code)
            curr_df = curr_df[curr_df['Active']]

            # create output path if it does not exist
            out_path = main_dir / "data" / "processed" / "players"
            out_path.mkdir(parents=True, exist_ok=True)

            # save output
            target_path = out_path / f"{cnt_code}_players.csv"
            curr_df.to_csv(target_path, index=False)
            print(f"\tData saved to {target_path}")
        except Exception as e:
            print(f"\t{e}")
    print('Done')

In [158]:
euro24_cnt_code_lst = euro24_country_df['IOC'].values.tolist()
collect_active_players_in_countries(euro24_cnt_code_lst)

	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\ALB_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\AUT_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\BEL_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\CRO_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\CZE_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\DEN_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\ENG_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\FRA_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\GEO_players.csv
	Data saved to D:\Users\Timothy\Projects\football-analysis\data\processed\players\GER_players.csv
	Data saved to D:\Us

### 2.3 Map by country
The Euro Fantasy data gives us the fixture which are 2 countries. So we try to check for each player, which country's active player list are they found in

In [3]:
def read_active_players_in_countries(country_lst, verbose=False):
    main_dir = Path.cwd().resolve().parents[0]
    out_path = main_dir / "data" / "processed" / "players"

    res = {}
    for cnt_code in country_lst:
        if verbose:
            print(f"Reading player data for country {cnt_code}")
        try:
            res[cnt_code] = pd.read_csv(out_path / f"{cnt_code}_players.csv")
            res[cnt_code].dropna(subset=['Name'], inplace=True)
            res[cnt_code]['name_deaccented'] = res[cnt_code]['Name'].apply(lambda x: unidecode(x))
        except Exception as e:
            print(f"\t{e}")
    return res

In [6]:
euro24_cnt_code_lst = euro24_country_df['IOC'].values.tolist()
country_df_map = read_active_players_in_countries(euro24_cnt_code_lst, verbose=True)

Reading player data for country ALB
Reading player data for country AUT
Reading player data for country BEL
Reading player data for country CRO
Reading player data for country CZE
Reading player data for country DEN
Reading player data for country ENG
Reading player data for country FRA
Reading player data for country GEO
Reading player data for country GER
Reading player data for country HUN
Reading player data for country ITA
Reading player data for country NED
Reading player data for country POL
Reading player data for country POR
Reading player data for country ROU
Reading player data for country SCO
Reading player data for country SRB
Reading player data for country SVK
Reading player data for country SVN
Reading player data for country ESP
Reading player data for country SUI
Reading player data for country TUR
Reading player data for country UKR


In [46]:
# def get_player_data_by_name_test(fe_df_row):
#     euro24_cnt_code_lst = euro24_country_df['IOC'].values.tolist()
#     country_df_map = read_active_players_in_countries(euro24_cnt_code_lst)

#     possible_countries = fe_df_row['Next Match'].split(' v ') # split fixture
#     name_regex = '^' + fe_df_row['regex_pattern']

#     res = []

#     for cnt_code in possible_countries:
#         cnt_code = cnt_code.upper()
#         if cnt_code in country_df_map:
#             df = country_df_map[cnt_code]
#             filt = df['name_deaccented'].str.contains(name_regex, case=False, regex=True)
#             res.append(filt.sum())
#     return res

# def get_player_data_by_name(fe_df_row):
#     euro24_cnt_code_lst = euro24_country_df['IOC'].values.tolist()
#     country_df_map = read_active_players_in_countries(euro24_cnt_code_lst)

#     possible_countries = fe_df_row['Next Match'].split(' v ') # split fixture
#     name_regex = '^' + fe_df_row['regex_pattern']

#     res_df = pd.DataFrame(columns=['Name', 'Country', 'url', 'Active', 'Years Played', 'Position', 'Clubs Played', 'IOC'])

#     for cnt_code in possible_countries:
#         cnt_code = cnt_code.upper()
#         if cnt_code in country_df_map:
#             df = country_df_map[cnt_code]
#             filt = df['name_deaccented'].str.contains(name_regex, case=False, regex=True)
#             if filt.sum() > 0:
#                 res_df = pd.concat([res_df, df[filt]], ignore_index=True)
#     return res_df

In [47]:
# df = fe_df.apply(get_player_data_by_name_test, axis=1)

In [78]:
# fe_df['possible_countries'] = fe_df['Next Match'].str.split(' v ') # split fixture
# df_left = fe_df.explode('possible_countries')
# print(df_left.shape)
# df_left.head()

In [77]:
# # concat all country dataframes from country_df_map
# df_right = None
# for cnt_code, df in country_df_map.items():
#     if df_right is None:
#         df_right = df
#     else:
#         df_right = pd.concat([df_right, df], ignore_index=True)
# print(df_right.shape)
# df_right.head()

(32927, 8)


,Name,url,Active,Years Played,Position,Clubs Played,IOC,name_deaccented
0,Kristal Abazaj,/en/players/c9bc334c/Kristal-Abazaj,True,2018-2023,FW,"Osijek, İstanbulspor",ALB,Kristal Abazaj
1,Alessio Abibi,/en/players/cd5069b8/Alessio-Abibi,True,2015-2023,GK,NaN,ALB,Alessio Abibi
2,Amir Abrashi,/en/players/1c08a3c8/Amir-Abrashi,True,2010-2024,MF,"Grasshopper, Albania, Freiburg, Basel",ALB,Amir Abrashi
3,Albion Ademi,/en/players/424471b0/Albion-Ademi,True,2016-2024,"FW,MF","Tianjin Jinmen Tiger, Inter Turku, Värnamo, IF...",ALB,Albion Ademi
4,Valon Ahmedi,/en/players/5d91cf0f/Valon-Ahmedi,True,2017-2020,"FW,MF","Inter Zaprešić, NK Maribor",ALB,Valon Ahmedi


In [84]:
fe_df.head()

,player_id,Players,Price,Total pts,Selected,MD pts,Pts per €,Pts per MD,PotM pts,Goals,...,Mins played,Trans in,Trans out,img_url,Next Match,Position,fname_init,lname,regex_pattern,keyword
0,0,K. Mbappé,11.0,0.0,76.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9715.0,2632.0,https://img.uefa.com/imgml/TP/players/3/2024/3...,AUT v FRA,FWD,K.,Mbappé,K.+ Mbappe,mb
1,1,H. Kane,11.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10142.0,4501.0,https://img.uefa.com/imgml/TP/players/3/2024/3...,SRB v ENG,FWD,H.,Kane,H.+ Kane,ka
2,2,C. Ronaldo,10.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3962.0,5013.0,https://img.uefa.com/imgml/TP/players/3/2024/3...,POR v CZE,FWD,C.,Ronaldo,C.+ Ronaldo,ro
3,3,J. Bellingham,9.5,0.0,50.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7235.0,6766.0,https://gaming.uefa.com/en/eurofantasy/static-...,SRB v ENG,MID,J.,Bellingham,J.+ Bellingham,be
4,4,K. De Bruyne,9.5,0.0,21.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5474.0,6343.0,https://gaming.uefa.com/en/eurofantasy/static-...,BEL v SVK,MID,K.,De Bruyne,K.+ De Bruyne,br


In [16]:
def simple_player_merge(df_left):
    euro24_cnt_code_lst = euro24_country_df['IOC'].values.tolist()
    country_df_map = read_active_players_in_countries(euro24_cnt_code_lst)

    # l_colnames = df_left.columns.values.tolist()
    r_colnames = ['Name', 'Country', 'url', 'Active', 'Years Played', 'Position', 'Clubs Played', 'IOC', 'index']
    res_df = pd.DataFrame(columns=r_colnames)
    
    for index, row in df_left.iterrows():
        is_changed = False

        possible_countries = row['Next Match'].split(' v ') # split fixture
        name_regex = '^' + row['regex_pattern']

        for cnt_code in possible_countries:
            cnt_code = cnt_code.upper()
            if cnt_code in country_df_map:
                df = country_df_map[cnt_code]
                filt = df['name_deaccented'].str.contains(name_regex, case=False, regex=True)
                if filt.sum() > 0:
                    is_changed = True
                    curr_df = df[filt]
                    curr_df['index'] = index
                    res_df = pd.concat([res_df, curr_df], ignore_index=True)
        if not is_changed:
            print(f"Could not find player data for {index} {row['Players']} in {possible_countries}")
        
    res_df = pd.merge(df_left, res_df, left_index=True, right_on='index', how='left')
    return res_df

In [17]:
final_df = simple_player_merge(fe_df)

Could not find player data for 33 M. Depay in ['POL', 'NED']
Could not find player data for 123 G. Sudakov in ['ROU', 'UKR']
Could not find player data for 230 Tosun in ['TUR', 'GEO']
Could not find player data for 238 V. Tsygankov in ['ROU', 'UKR']
Could not find player data for 348 J. Brunn Larsen in ['SVN', 'DEN']
Could not find player data for 358 O. Pikhalonok in ['ROU', 'UKR']
Could not find player data for 578 G. Tsitaishvili in ['TUR', 'GEO']
Could not find player data for 586 Uçan in ['TUR', 'GEO']
Could not find player data for 593 I. Konoplianka in ['ROU', 'UKR']
Could not find player data for 651 S. Lobjanidze in ['TUR', 'GEO']
Could not find player data for 655 V. Vladimer in ['TUR', 'GEO']
Could not find player data for 715 O. Karavaiev in ['ROU', 'UKR']
Could not find player data for 801 T. Keher in ['GER', 'SCO']
Could not find player data for 815 G. Gocholeishvilli in ['TUR', 'GEO']
Could not find player data for 823 G. Gvelesiani in ['TUR', 'GEO']
Could not find playe

In [31]:
final_df.to_csv("../data/processed/euro24_players_country_draft.csv", index=False)

```# Sample DataFrames
A = pd.DataFrame({
    'regex': [r'^foo.*', r'^bar.*'],
    'anotherColumn': [1, 2],
    'A_data': ['data1', 'data2']
})

B = pd.DataFrame({
    'name': ['foobar', 'bazbar', 'foobaz', 'barqux'],
    'anotherColumn': [1, 2, 3, 2],
    'B_data': ['dataA', 'dataB', 'dataC', 'dataD']
})

# Step 1: Create a boolean mask for regex matching
A['key'] = 1  # Temporary key for cross join
B['key'] = 1  # Temporary key for cross join

# Perform cross join
cross_joined = pd.merge(A, B, on='key').drop('key', axis=1)

# Apply regex condition
cross_joined['regex_match'] = cross_joined.apply(lambda row: pd.notnull(row['name']) and pd.notnull(row['regex']) and bool(pd.Series(row['name']).str.match(row['regex']).iloc[0]), axis=1)

# Step 2: Filter based on both conditions
filtered = cross_joined[(cross_joined['regex_match']) & (cross_joined['anotherColumn_x'] == cross_joined['anotherColumn_y'])]

# Step 3: Select and rename columns as needed
result = filtered[['regex', 'anotherColumn_x', 'A_data', 'name', 'B_data']]
result.columns = ['regex', 'anotherColumn', 'A_data', 'name', 'B_data']

print(result)```